<a href="https://colab.research.google.com/github/RapidAI/RapidOCR/blob/main/RapidOCRDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## RapidOCR Python Demo



#### Set up

In [12]:
!git clone https://github.com/RapidAI/RapidOCR.git
%cd RapidOCR/python

Cloning into 'RapidOCR'...
remote: Enumerating objects: 3447, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (299/299), done.
remote: Total 3447 (delta 191), reused 228 (delta 77), pack-reused 3050
Receiving objects: 100% (3447/3447), 13.93 MiB | 23.69 MiB/s, done.
Resolving deltas: 100% (1563/1563), done.
/content/RapidOCR/python/RapidOCR/python


In [13]:
!pip install -r requirements.txt -i https://pypi.douban.com/simple/

Looking in indexes: https://pypi.douban.com/simple/


In [14]:
!wget --no-check-certificate "https://docs.google.com/uc?export=download&id=1MurIeU78qIGN8Jb2dK2_mnmEsEHH0Jct" -O models.zip
!rm -r models/
!unzip models.zip && rm models.zip

--2021-12-18 02:57:05--  https://docs.google.com/uc?export=download&id=1MurIeU78qIGN8Jb2dK2_mnmEsEHH0Jct
Resolving docs.google.com (docs.google.com)... 172.217.164.142, 2607:f8b0:4004:814::200e
Connecting to docs.google.com (docs.google.com)|172.217.164.142|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0rm8hfqhdh2brc42fbgdpmndj0h54gss/1639796175000/10158993189489619361/*/1MurIeU78qIGN8Jb2dK2_mnmEsEHH0Jct?e=download [following]
--2021-12-18 02:57:11--  https://doc-00-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0rm8hfqhdh2brc42fbgdpmndj0h54gss/1639796175000/10158993189489619361/*/1MurIeU78qIGN8Jb2dK2_mnmEsEHH0Jct?e=download
Resolving doc-00-3s-docs.googleusercontent.com (doc-00-3s-docs.googleusercontent.com)... 172.217.2.97, 2607:f8b0:4004:80a::2001
Connecting to doc-00-3s-docs.googleusercontent.com (doc-00-3s-docs.googl

## Run as shell script

In [15]:
!bash rapidOCR.sh

Traceback (most recent call last):
  File "rapidOCR.py", line 295, in <module>
    keys_path=args.keys_path)
  File "rapidOCR.py", line 154, in __init__
    self.text_detector = TextDetector(det_model_path)
  File "/content/RapidOCR/python/RapidOCR/python/ch_ppocr_mobile_v2_det/text_detect.py", line 62, in __init__
    self.session = onnxruntime.InferenceSession(det_model_path, sess_opt)
  File "/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 280, in __init__
    self._create_inference_session(providers, provider_options)
  File "/usr/local/lib/python3.7/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py", line 307, in _create_inference_session
    sess = C.InferenceSession(session_options, self._model_path, True, self._read_config_from_model)
onnxruntime.capi.onnxruntime_pybind11_state.NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from models/ch_ppocr_mobile_v2.0_det_infer.onnx failed:Load model models/

## Run as code

In [19]:
import copy
import math
import random
from pathlib import Path

import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

from ch_ppocr_mobile_v2_cls import TextClassifier
from ch_ppocr_mobile_v2_det import TextDetector
from ch_ppocr_mobile_v2_rec import TextRecognizer

In [20]:
def draw_ocr_box_txt(image, boxes, txts,
                     scores=None, text_score=0.5,
                     font_path="./models/msyh.ttc"):
    h, w = image.height, image.width
    img_left = image.copy()
    img_right = Image.new('RGB', (w, h), (255, 255, 255))

    random.seed(0)
    draw_left = ImageDraw.Draw(img_left)
    draw_right = ImageDraw.Draw(img_right)
    for idx, (box, txt) in enumerate(zip(boxes, txts)):
        if scores is not None and scores[idx] < text_score:
            continue

        color = (random.randint(0, 255),
                 random.randint(0, 255),
                 random.randint(0, 255))
        draw_left.polygon(box, fill=color)
        draw_right.polygon([box[0][0], box[0][1],
                            box[1][0], box[1][1],
                            box[2][0], box[2][1],
                            box[3][0], box[3][1]],
                           outline=color)

        box_height = math.sqrt((box[0][0] - box[3][0])**2
                               + (box[0][1] - box[3][1])**2)

        box_width = math.sqrt((box[0][0] - box[1][0])**2
                              + (box[0][1] - box[1][1])**2)

        if box_height > 2 * box_width:
            font_size = max(int(box_width * 0.9), 10)
            font = ImageFont.truetype(font_path, font_size,
                                      encoding="utf-8")
            cur_y = box[0][1]
            for c in txt:
                char_size = font.getsize(c)
                draw_right.text((box[0][0] + 3, cur_y), c,
                                fill=(0, 0, 0), font=font)
                cur_y += char_size[1]
        else:
            font_size = max(int(box_height * 0.8), 10)
            font = ImageFont.truetype(font_path, font_size,
                                      encoding="utf-8")
            draw_right.text([box[0][0], box[0][1]], txt,
                            fill=(0, 0, 0), font=font)

    img_left = Image.blend(image, img_left, 0.5)
    img_show = Image.new('RGB', (w * 2, h), (255, 255, 255))
    img_show.paste(img_left, (0, 0, w, h))
    img_show.paste(img_right, (w, 0, w * 2, h))
    return np.array(img_show)


def visualize(image_path, boxes, rec_res):
    image = Image.open(image_path)
    txts = [rec_res[i][0] for i in range(len(rec_res))]
    scores = [rec_res[i][1] for i in range(len(rec_res))]

    draw_img = draw_ocr_box_txt(image, boxes,
                                txts, scores,
                                text_score=0.5)

    draw_img_save = Path("./inference_results/")
    if not draw_img_save.exists():
        draw_img_save.mkdir(parents=True, exist_ok=True)

    image_save = str(draw_img_save / f'infer_{Path(image_path).name}')
    cv2.imwrite(image_save, draw_img[:, :, ::-1])
    print(f'The infer result has saved in {image_save}')


class TextSystem(object):
    def __init__(self, det_model_path,
                    rec_model_path,
                    use_angle_cls=False,
                    cls_model_path=None,
                    keys_path=None):
        super(TextSystem).__init__()
        self.text_detector = TextDetector(det_model_path)
        self.text_recognizer = TextRecognizer(rec_model_path, keys_path)
        self.use_angle_cls = use_angle_cls
        if self.use_angle_cls:
            self.text_classifier = TextClassifier(cls_model_path)
        self.text_score = 0.5

    def get_rotate_crop_image(self, img, points):
        '''
        img_height, img_width = img.shape[0:2]
        left = int(np.min(points[:, 0]))
        right = int(np.max(points[:, 0]))
        top = int(np.min(points[:, 1]))
        bottom = int(np.max(points[:, 1]))
        img_crop = img[top:bottom, left:right, :].copy()
        points[:, 0] = points[:, 0] - left
        points[:, 1] = points[:, 1] - top
        '''
        img_crop_width = int(
            max(
                np.linalg.norm(points[0] - points[1]),
                np.linalg.norm(points[2] - points[3])))
        img_crop_height = int(
            max(
                np.linalg.norm(points[0] - points[3]),
                np.linalg.norm(points[1] - points[2])))
        pts_std = np.float32([[0, 0], [img_crop_width, 0],
                                [img_crop_width, img_crop_height],
                                [0, img_crop_height]])
        M = cv2.getPerspectiveTransform(points, pts_std)
        dst_img = cv2.warpPerspective(
            img,
            M, (img_crop_width, img_crop_height),
            borderMode=cv2.BORDER_REPLICATE,
            flags=cv2.INTER_CUBIC)
        dst_img_height, dst_img_width = dst_img.shape[0:2]
        if dst_img_height * 1.0 / dst_img_width >= 1.5:
            dst_img = np.rot90(dst_img)
        return dst_img

    @staticmethod
    def sorted_boxes(dt_boxes):
        """
        Sort text boxes in order from top to bottom, left to right
        args:
            dt_boxes(array):detected text boxes with shape [4, 2]
        return:
            sorted boxes(array) with shape [4, 2]
        """
        num_boxes = dt_boxes.shape[0]
        sorted_boxes = sorted(dt_boxes, key=lambda x: (x[0][1], x[0][0]))
        _boxes = list(sorted_boxes)

        for i in range(num_boxes - 1):
            if abs(_boxes[i + 1][0][1] - _boxes[i][0][1]) < 10 and \
                    (_boxes[i + 1][0][0] < _boxes[i][0][0]):
                tmp = _boxes[i]
                _boxes[i] = _boxes[i + 1]
                _boxes[i + 1] = tmp
        return _boxes

    @staticmethod
    def load_image(image_path):
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"error in loading image:{image_path}")
        return img

    def __call__(self, image_path):
        img = self.load_image(image_path)
        dt_boxes, elapse = self.text_detector(img)
        print("dt_boxes num : {}, elapse : {}".format(
            len(dt_boxes), elapse))
        if dt_boxes is None or len(dt_boxes) < 1:
            return None, None
        dt_boxes = self.sorted_boxes(dt_boxes)

        img_crop_list = []
        for bno in range(len(dt_boxes)):
            tmp_box = copy.deepcopy(dt_boxes[bno])
            img_crop = self.get_rotate_crop_image(img, tmp_box)
            img_crop_list.append(img_crop)

        if self.use_angle_cls:
            img_crop_list, angle_list, elapse = self.text_classifier(
                img_crop_list)
            print("cls num  : {}, elapse : {}".format(
                len(img_crop_list), elapse))

        rec_res, elapse = self.text_recognizer(img_crop_list)
        print("rec_res num  : {}, elapse : {}".format(
            len(rec_res), elapse))

        filter_boxes, filter_rec_res = [], []
        for box, rec_reuslt in zip(dt_boxes, rec_res):
            text, score = rec_reuslt
            if score >= self.text_score:
                filter_boxes.append(box)
                filter_rec_res.append(rec_reuslt)
        return filter_boxes, filter_rec_res

In [21]:
det_model_path = "models/ch_PP-OCRv2_det_infer.onnx"
rec_model_path = "models/ch_ppocr_mobile_v2.0_rec_infer.onnx"
cls_model_path = "models/ch_ppocr_mobile_v2.0_cls_infer.onnx"

keys_path = "ch_ppocr_mobile_v2_rec/ppocr_keys_v1.txt"

text_sys = TextSystem(det_model_path, 
            rec_model_path, 
            use_angle_cls=True, 
            cls_model_path=cls_model_path, 
            keys_path=keys_path)

In [22]:
img_path = '/content/RapidOCR/images/1.jpg'
dt_boxes, rec_res = text_sys(img_path)
print(rec_res)
visualize(img_path, dt_boxes, rec_res)

dt_boxes num : 18, elapse : 0.3534584045410156
cls num  : 18, elapse : 0.03909873962402344
rec_res num  : 18, elapse : 0.14548611640930176
The infer result has saved in inference_results/infer_1.jpg
